In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import time

In [6]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
#https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={start_date}&de={end_date}&start={page}
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:망간 전기차 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):50

크롤링할 종료 페이지:  50 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=망간 전기차 배터리&pd=3&

100%|███████████████████████████████████████████| 47/47 [00:09<00:00,  4.94it/s]

검색된 기사 갯수: 총  500 개

[뉴스 제목]
['"10 Minutes" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑', '엑셀 밟는 비야디 전기차…"연내 승용차 브랜드 론칭"', "'3천만원대' 기아 EV3 6월 출시…전기차 '캐즘' 돌파 선봉장", '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', '현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다', "현대차·기아, CFO '전기차 동상이몽'...캐스퍼·EV3 전략 온도차", '車업계, 전기차 신차 잇단 출시…캐즘 정면돌파 의지', '[대한민국 100대 CEO] 최윤호 삼성SDI 대표이사 사장 | 전기차 ‘캐즘’ 덫? 질적 성장 순항 중', "SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상", '[인터뷰] 벤츠 최고기술책임자 "전동화전략 여전…전기차 할인경쟁 안해"', "전략특허 58% 뚫렸다…뿔난 LG엔솔, '배터리 특허 전쟁' 선언", '[단독] 현대차 ‘아이오닉9’에 SK온 배터리 탑재된다', 'CATL 호실적에 밀린 K-배터리 3사…“中 이길 대안은 기술력·라인업 다변화” [비즈360]', '[배터리완전정복](33)中 LFP 대적할 3대 키워드…단결정·고전압·미드니켈', 'CATL “볼보 전기차 사용 후 배터리 재활용”', 'SK온 코발트 프리 배터리·LG이노텍 연속줌 카메라, 美발명상 에디슨어워즈 수상', '배터리 재활용·투명한 공급망…‘지속가능 경영’', "삼성SDI, 전기차용 LFP 배터리 전략 확대…'LMFP→LFP+'", '[르포] 배터리 소재 ‘블랙 매스’ 연 5000t 만드는 DS단석', '[보죠, 배터리]저가 광물 시대, 양극재 기업의 생존법', '배터리 對中 적자 해법 있나…"기술·가격 협상 우위 없어"', '폰·노트북 배터리 수명 2배 늘리는 충전 기술 나왔다', '기아 “전기차 성장세 둔화, 하이브리드차 판매 확대로 대응”', "'중국산 모델3' 저가 공세…국내

In [3]:
#df1 = pd.read_csv('니켈 배터리_20240626_02시00분37초.csv')#망간 배터리
df1= news_df.copy()

In [5]:
#df2 = pd.read_csv('니켈 원자재_20240626_02시01분47초.csv') #망간 원자재
df2= news_df.copy()

In [7]:
#df3 = pd.read_csv('니켈 전기차 배터리_20240626_02시02분31초.csv') #망간 전기차 배터리
df3= news_df.copy()

In [8]:
import re

def clean_text(text):
    # HTML 태그 제거
    text = re.sub('<[^<]+?>', '', text)
    # 줄바꿈, 탭 문자 제거
    text = re.sub('[\r\n\t]', ' ', text)
    # 특수문자 제거 (예: ', ", -, ...)
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)
    # 중복 공백 제거
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [9]:
df1['Content'] = df1['content'].apply(clean_text)
df1 = df1[['date','title','Content']]
df2['Content'] = df2['content'].apply(clean_text)
df2 = df2[['date','title','Content']]
df3['Content'] = df3['content'].apply(clean_text)
df3 = df3[['date','title','Content']]

In [10]:
df1['Keyword'] = '망간 배터리'
df2['Keyword'] = '망간 원자재'
df3['Keyword'] = '망간 전기차 배터리'

In [11]:
df1

,date,title,Content,Keyword
0,2024-04-30 14:26:01,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 320까지...,망간 배터리
1,2024-04-29 06:20:00,"""배터리 가격 경쟁력 갖추자""…이석희 SK온 사장 '승부수'",글로벌 전기차용 배터리 사용량그래픽임종철배터리 가격 경쟁력 확보에 주력해야 한다28...,망간 배터리
2,2024-04-29 15:56:01,"K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진",지난달 서울 강남구 코엑스에서 열린 인터배터리 2024를 찾은 관람객들이 SK온 배...,망간 배터리
3,2024-04-30 11:30:01,현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다,현대차SK온 신차에 공급 협의SK온 서산공장 증설도 본격 돌입 SK온 서산 배터리공...,망간 배터리
4,2024-04-21 09:22:01,"SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상",기술력amp지속가능성 우수 평가SK온은 자사 코발트 프리 배터리가 지난 18일 미국...,망간 배터리
5,2024-04-24 18:17:01,[단독] 현대차 ‘아이오닉9’에 SK온 배터리 탑재된다,아이오닉9 연내 출시 앞둔 대형 플래그십 SUV 현대차SK온 배터리 협력 가속도SK...,망간 배터리
6,2024-04-24 17:32:08,CATL 호실적에 밀린 K-배터리 3사…“中 이길 대안은 기술력·라인업 다변화” [...,CATL 전기차 캐즘 에도 1분기 순익 2조원K배터리 3사 실적 하락세에 고전 장기...,망간 배터리
7,2024-04-25 06:00:00,"전략특허 58% 뚫렸다…뿔난 LG엔솔, '배터리 특허 전쟁' 선언",LG에너지솔루션 특허 현황그래픽조수아LG에너지솔루션이 특허 무임승차에 대한 무관용 ...,망간 배터리
8,2024-04-25 13:59:01,"""배터리, 더이상 일회용품 아냐…소듐·전고체·리튬황 배터리 부상""",25일 한국화학공학회 봄 학술대회 셜리 멍 미국 시카고대 교수 기조강연 셜리 멍 미...,망간 배터리
9,2024-04-20 08:00:00,[배터리완전정복](33)中 LFP 대적할 3대 키워드…단결정·고전압·미드니켈,편집자주지금은 배터리 시대입니다 휴대폰노트북전기자동차 등 거의 모든 곳에 배터리가 ...,망간 배터리


In [12]:
# 검색 키워드 리스트
keywords = ['망간 배터리', '망간 원자재', '망간 전기차 배터리']

In [13]:
# 데이터프레임을 리스트로 묶어서 concat 함수에 전달하여 합침
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,date,title,Content,Keyword
0,2024-04-30 14:26:01,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 320까지...,망간 배터리
1,2024-04-29 06:20:00,"""배터리 가격 경쟁력 갖추자""…이석희 SK온 사장 '승부수'",글로벌 전기차용 배터리 사용량그래픽임종철배터리 가격 경쟁력 확보에 주력해야 한다28...,망간 배터리
2,2024-04-29 15:56:01,"K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진",지난달 서울 강남구 코엑스에서 열린 인터배터리 2024를 찾은 관람객들이 SK온 배...,망간 배터리
3,2024-04-30 11:30:01,현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다,현대차SK온 신차에 공급 협의SK온 서산공장 증설도 본격 돌입 SK온 서산 배터리공...,망간 배터리
4,2024-04-21 09:22:01,"SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상",기술력amp지속가능성 우수 평가SK온은 자사 코발트 프리 배터리가 지난 18일 미국...,망간 배터리
...,...,...,...,...
122,2024-04-08 12:01:44,"보급형·중저가 앞세워… 중국, 배터리 생산 점유율도 74%[한국경제 흔드는 ‘차이나...",CATL 시장점유율 38K배터리 3보다 더높아 저가의 중국산 리튬인산철LFP 배터리...,망간 전기차 배터리
123,2024-04-08 05:31:26,부진하는 배터리·유화…재료 수입도 줄었다,경쟁 심화 등 업황 악화12월 주요소재 수입액 12억 유화건설 경기 부진 영향나프타...,망간 전기차 배터리
124,2024-04-10 16:32:07,"성일하이텍, LFP 배터리도 재활용한다…파일럿 라인 내년 1분기 가동",성일하이텍 군산 공장 전경 사진성일하이텍성일하이텍이 리튬인산철LFP 배터리 재활용을...,망간 전기차 배터리
125,2024-03-31 23:09:02,[설왕설래] 애플 제친 샤오미 전기차,역대 노벨상 수상자 중 최연장자인 존 굿이너프 박사는 오늘날 전 세계 전기차EV에 ...,망간 전기차 배터리


In [14]:
df.to_csv('df_망간2.csv', index = False)

In [19]:
# 한국어 불용어 리스트 (필요에 따라 추가/제거 가능)
korean_stopwords = ['것으로', '올해','있는','대한','이후','지난해','위한',
    '이', '그', '저', '것', '있', '하', '것들', '들', '에서', '의', '를', '으로', '에', '그리고', '가', '를', '에게',
    '뿐만', '뿐', '이다', '있다', '없다', '에서', '과', '도', '을', '와', '이다', '으로', '이다', '것이다', '더', '등',
    '만', '이다', '또', '한', '가장', '것', '때문에', '로', '에', '이', '들', '잘', '한', '수', '하다'
]


In [20]:
# TF-IDF 분석 및 중요 단어 추출
def tfidf_analysis(df, keywords):
    results = {}
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=korean_stopwords)
    
    for keyword in keywords:
        keyword_df = df[df['Keyword'] == keyword].reset_index(drop=True)
        if not keyword_df.empty:
            tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_df['Content'])
            feature_names = tfidf_vectorizer.get_feature_names_out()
            tfidf_df = pd.DataFrame(tfidf_matrix.T.toarray(), index=feature_names)
            tfidf_df.columns = keyword_df['title']
            
            # 중요 단어 추출
            keyword_tfidf = tfidf_df.sum(axis=1)
            top_terms = keyword_tfidf.sort_values(ascending=False).head(10)
            
            results[keyword] = top_terms
    
    return results

In [21]:
# TF-IDF 분석
tfidf_results = tfidf_analysis(df, keywords)

In [22]:
# 결과 출력
for keyword, top_terms in tfidf_results.items():
    print(f"\n키워드: {keyword}\n")
    print(top_terms)


키워드: 망간 배터리

배터리     6.105466
전기차     3.726768
lfp     3.226410
재활용     3.193677
배터리를    2.298652
중국      1.709907
국내      1.654948
글로벌     1.543102
미국      1.515271
배터리는    1.476898
dtype: float64

키워드: 망간 원자재

배터리     4.459405
전기차     3.081660
양극재     2.147692
중국      1.850768
lfp     1.715243
가격      1.481444
국내      1.361741
배터리를    1.285824
보조금     1.277159
가격이     1.238434
dtype: float64

키워드: 망간 전기차 배터리

배터리     5.565125
전기차     4.001998
lfp     2.895164
재활용     2.350623
배터리를    2.205780
국내      1.758175
중국      1.715214
글로벌     1.651382
충전      1.406196
배터리는    1.399394
dtype: float64


In [19]:
# 모든 DataFrame 및 분석 결과 저장
df.to_csv('all_news_articles.csv', index=False)
for keyword in keywords:
    keyword_df = df[df['Keyword'] == keyword]
    keyword_df.to_csv(f'news_articles_{keyword}.csv', index=False)
    if keyword in tfidf_results:
        tfidf_results[keyword].to_csv(f'top_terms_{keyword}.csv')

# 망간 최종 키워드 : 망간 전기차 배터리

In [21]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
#https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={start_date}&de={end_date}&start={page}
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:망간 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):400

크롤링할 종료 페이지:  400 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=망간 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=

100%|███████████████████████████████████████████| 78/78 [00:13<00:00,  5.85it/s]

검색된 기사 갯수: 총  4000 개

[뉴스 제목]
['"10 Minutes" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑', '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', '현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다', "SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상", '[단독] 현대차 ‘아이오닉9’에 SK온 배터리 탑재된다', 'CATL 호실적에 밀린 K-배터리 3사…“中 이길 대안은 기술력·라인업 다변화” [비즈360]', "전략특허 58% 뚫렸다…뿔난 LG엔솔, '배터리 특허 전쟁' 선언", '"배터리, 더이상 일회용품 아냐…소듐·전고체·리튬황 배터리 부상"', '[배터리완전정복](33)中 LFP 대적할 3대 키워드…단결정·고전압·미드니켈', '폐보조배터리서 희귀 금속 추출… 자원 재순환 확산 이끈다 [연중기획-대한민국 ESG 경영 리포트]', 'SK온 코발트 프리 배터리·LG이노텍 연속줌 카메라, 美발명상 에디슨어워즈 수상', "삼성SDI, 전기차용 LFP 배터리 전략 확대…'LMFP→LFP+'", '배터리 對中 적자 해법 있나…"기술·가격 협상 우위 없어"', '배터리 재활용·투명한 공급망…‘지속가능 경영’', '현대차, 인도 전기차 생산 준비…현지 배터리 쓴다', '현대차·기아, 인도산 배터리 탑재 전기차 생산한다', "DS단석, 첫 리튬이온배터리 재활용공장 준공…'블랙매스' 연산 5000톤", 'CATL “볼보 전기차 사용 후 배터리 재활용”', '김종완 DS단석 대표 "재활용 100% 전구체 만든다"…\'70조 시장\' 정조준', '[보죠, 배터리]저가 광물 시대, 양극재 기업의 생존법', '폰·노트북 배터리 수명 2배 늘리는 충전 기술 나왔다', '테슬라 빼곤 가격 인하 노력이 안 보인다', '배터리 약세에…LG에너지솔루션, ‘도움닫기’ 빼면 상장 후 첫 적자(종합)', 'SK배터리아메리카, 美 노동부서 1억원 과

In [22]:
news_df

,date,title,link,content
0,2024-04-30 14:26:01,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",https://n.news.naver.com/mnews/article/018/000...,[\n주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 32...
1,2024-04-29 06:20:00,"""배터리 가격 경쟁력 갖추자""…이석희 SK온 사장 '승부수'",https://n.news.naver.com/mnews/article/008/000...,"[\n\n\n\n\n글로벌 전기차용 배터리 사용량/그래픽=임종철""배터리 가격 경쟁력..."
2,2024-04-29 15:56:01,"K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진",https://n.news.naver.com/mnews/article/032/000...,[\n\n\n\n\n지난달 서울 강남구 코엑스에서 열린 ‘인터배터리 2024’를 찾...
3,2024-04-30 11:30:01,현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다,https://n.news.naver.com/mnews/article/016/000...,"[\n현대차·SK온, 신차에 공급 협의SK온 서산공장 증설도 본격 돌입\n\n\n\..."
4,2024-04-21 09:22:01,"SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상",https://n.news.naver.com/mnews/article/092/000...,"[\n""기술력&amp;지속가능성 우수 평가""SK온은 자사 코발트 프리 배터리가 지난..."
...,...,...,...,...
67,2024-04-17 11:02:11,1분기 양극재 수출 회복세,https://n.news.naver.com/mnews/article/030/000...,[\n\n\n\n\n에코프로 하이니켈 양극재 (에코프로 제공)지난해 4분기 월 1만...
68,2024-03-05 00:11:02,연초 무역 청신호 켜졌다?…대중 흑자는 ‘통계 착시’,https://n.news.naver.com/mnews/article/025/000...,[\n\t\t\t 연초 수출 성적표에 일단 청신호가 켜졌다. 대(對) 중국 무역수...
69,2024-03-04 16:31:11,수출 5개월 연속 늘었지만…대중국 무역흑자 '불안한 반전' 왜,https://n.news.naver.com/mnews/article/025/000...,[\n\t\t\t 연초 수출 성적표에 일단 청신호가 켜졌다. 대(對) 중국 무역수...
70,2024-03-06 13:16:13,NCM의 장점은?,https://n.news.naver.com/mnews/article/001/001...,[\n\n\n\n\n (서울=연합뉴스) 김도훈 기자 = 6일 서울 강남구 코엑...


In [20]:
df1 = news_df.copy() #100행

In [14]:
df2 = news_df.copy() #200행

In [17]:
df3 = news_df.copy() #300행

In [23]:
df4 = news_df.copy() #400행

In [24]:
# 각 DataFrame의 행 개수를 구하기
row_count_df1 = len(df1)
row_count_df2 = len(df2)
row_count_df3 = len(df3)
row_count_df4 = len(df4)

# 가장 많은 행 수를 가진 DataFrame을 출력
max_rows_df = max([(row_count_df1, 'df1'), (row_count_df2, 'df2'), (row_count_df3, 'df3'), (row_count_df4, 'df4')])[1]
print(f"가장 많은 행 수를 가진 DataFrame은 {max_rows_df}입니다.")

print(df1.shape, df2.shape, df3.shape, df4.shape)

가장 많은 행 수를 가진 DataFrame은 df3입니다.
(54, 4) (45, 4) (93, 4) (72, 4)


In [25]:
# 1. pDate 열을 datetime 형식으로 변환
df3['date'] = pd.to_datetime(df3['date'])
# 2. pDate 열을 기준으로 데이터프레임을 날짜순으로 정렬
df3 = df3.sort_values(by='date')

df3.to_csv('망간_뉴스크롤링_최종.csv', index = False)
